In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import random
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from datetime import datetime

# Dummy dataset simulation for sign language gestures (5 different signs)
def generate_dummy_data():
    signs = ['Hello', 'ThankYou', 'Yes', 'No', 'Goodbye']
    data = []
    for i in range(100):  # Generate 100 images
        sign = random.choice(signs)
        image = np.random.rand(64, 64, 3)  # Random image data (64x64 pixels, 3 color channels)
        data.append((image, sign))
    return data

# CNN model creation for sign language gesture classification
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(5, activation='softmax'))  # 5 outputs: 'Hello', 'ThankYou', 'Yes', 'No', 'Goodbye'

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Dummy prediction function for testing
def predict_sign_language(model, image):
    # Simulate prediction by randomly selecting one of the signs
    signs = ['Hello', 'ThankYou', 'Yes', 'No', 'Goodbye']
    return random.choice(signs)
 
# Operational Time Check (6 PM to 10 PM)
def is_operational_time():
    current_hour = datetime.now().hour
    return 18 <= current_hour <= 22  # Between 6 PM and 10 PM

# GUI setup for Sign Language Detection
class SignLanguageDetector(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title('Sign Language Detection')
        self.geometry('600x600')

        self.model = create_model()  # Load your trained model
        self.model.summary()  # Show the model summary in the console

        self.image_label = tk.Label(self, text="Upload an image or use video for sign language detection")
        self.image_label.pack()

        self.upload_button = tk.Button(self, text="Upload Image", command=self.upload_image)
        self.upload_button.pack()

        self.result_label = tk.Label(self, text="Results will be shown here.", justify=tk.LEFT)
        self.result_label.pack()

        self.video_button = tk.Button(self, text="Start Video Stream", command=self.start_video_stream)
        self.video_button.pack()

    def upload_image(self):
        if not is_operational_time():
            messagebox.showinfo("Error", "The system is only operational between 6 PM and 10 PM.")
            return

        file_path = filedialog.askopenfilename(title="Select an Image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if file_path:
            image = Image.open(file_path)
            image = image.resize((200, 200))
            img_tk = ImageTk.PhotoImage(image)

            self.image_label.config(image=img_tk, text="")
            self.image_label.image = img_tk

            sign = predict_sign_language(self.model, file_path)
            self.result_label.config(text=f"Predicted Sign Language Gesture: {sign}")

    def start_video_stream(self):
        if not is_operational_time():
            messagebox.showinfo("Error", "The system is only operational between 6 PM and 10 PM.")
            return

        self.capture = cv2.VideoCapture(0)
        self.video_stream()

    def video_stream(self):
        ret, frame = self.capture.read()
        if ret:
            # Resize the frame for prediction
            frame_resized = cv2.resize(frame, (64, 64))
            frame_resized = np.expand_dims(frame_resized, axis=0)  # Add batch dimension

            # Convert the frame to a displayable image for Tkinter
            cv2_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(cv2_image)
            img = img.resize((300, 300))
            img_tk = ImageTk.PhotoImage(img)

            # Display the image in the Tkinter window
            self.image_label.config(image=img_tk, text="")
            self.image_label.image = img_tk

            # Simulate prediction
            sign = predict_sign_language(self.model, frame_resized)
            self.result_label.config(text=f"Predicted Sign: {sign}")

            # Loop the video stream
            self.after(10, self.video_stream)
        else:
            self.capture.release()

# Run the GUI application
if __name__ == "__main__":
    app = SignLanguageDetector()
    app.mainloop()


c:\Users\Shweta\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,625,797 (6.20 MB)

 Trainable params: 1,625,797 (6.20 MB)

 Non-trainable params: 0 (0.00 B)